In [27]:
import geopandas as gpd # for geospatial data handling
import pandas as pd
import osmnx as ox # for handling data from OpenStreetMap (osm) with the help of networkX (nx)
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting
from pyproj import CRS # for more advanced CRS modifications and transformations
from shapely.geometry import Polygon

In [11]:
# Define the city name or bounding box
city = 'Montreal, Canada'

In [4]:
pois = ox.geometries_from_place(
    city, tags={"amenity": True}
)

/opt/conda/lib/python3.9/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
/opt/conda/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/opt/conda/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [12]:
len(pois)

25615

In [6]:
len(pois['amenity'].value_counts())

131

In [8]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the full list of value counts for the 'amenity' column
print(pois['amenity'].value_counts())

parking                    4827
bench                      4333
bicycle_parking            3134
restaurant                 1887
waste_basket               1116
parking_space               795
charging_station            778
cafe                        739
fast_food                   580
school                      562
post_box                    465
place_of_worship            453
drinking_water              422
taxi                        350
loading_dock                313
parking_entrance            296
vending_machine             275
bar                         265
bank                        262
social_facility             231
pharmacy                    222
telephone                   222
shelter                     208
toilets                     189
fuel                        180
bicycle_rental              176
dentist                     143
kindergarten                141
childcare                   122
clinic                      120
fountain                    110
communit

In [34]:
pois.head()

direction                    geometry     amenity  \
element_type osmid                                                         
node         189770160       NaN  POINT (-73.66345 45.54186)  restaurant   
             190646563       NaN  POINT (-73.66300 45.54201)    post_box   
             191024110       NaN  POINT (-73.67232 45.54761)  restaurant   
             191024149       NaN  POINT (-73.67179 45.54839)  restaurant   
             191024164       NaN  POINT (-73.67196 45.54923)  restaurant   

                                                        name    created_by  \
element_type osmid                                                           
node         189770160                         GP Restaurant           NaN   
             190646563                                   NaN  Potlatch 0.6   
             191024110                            Pizza Roni          JOSM   
             191024149                           La Crèmière           NaN   
             191024164  Le Palais du Chausson et de la Pizza           NaN   

                       addr:city addr:housenumber addr:postcode addr:province  \
element_type osmid                                                              
node         189770160       NaN              NaN           NaN           NaN   
             190646563       NaN              NaN           NaN           NaN   
             191024110       NaN              NaN           NaN           NaN   
             191024149       NaN              NaN           NaN           NaN   
             191024164       NaN              NaN           NaN           NaN   

                       addr:street brand brand:wikidata operator  \
element_type osmid                                                 
node         189770160         NaN   NaN            NaN      NaN   
             190646563         NaN   NaN            NaN      NaN   
             191024110         NaN   NaN            NaN      NaN   
             191024149         NaN   NaN            NaN      NaN   
             191024164         NaN   NaN            NaN      NaN   

                       operator:wikidata automatic_door  \
element_type osmid                                        
node         189770160               NaN            NaN   
             190646563               NaN            NaN   
             191024110               NaN            NaN   
             191024149               NaN            NaN   
             191024164               NaN            NaN   

                       check_date:opening_hours entrance opening_hours  \
element_type osmid                                                       
node         189770160                      NaN      NaN           NaN   
             190646563                      NaN      NaN           NaN   
             191024110                      NaN      NaN           NaN   
             191024149                      NaN      NaN           NaN   
             191024164                      NaN      NaN           NaN   

                       toilets:wheelchair wheelchair source brand:wikipedia  \
element_type osmid                                                            
node         189770160                NaN        NaN    NaN             NaN   
             190646563                NaN        NaN    NaN             NaN   
             191024110                NaN        NaN    NaN             NaN   
             191024149                NaN        NaN    NaN             NaN   
             191024164                NaN        NaN    NaN             NaN   

                       is_in:city name:en old_name phone website addr:unit  \
element_type osmid                                                           
node         189770160        NaN     NaN      NaN   NaN     NaN       NaN   
             190646563        NaN     NaN      NaN   NaN     NaN       NaN   
             191024110        NaN     NaN      NaN   NaN     NaN       NaN   
             19102414

In [36]:
# remove any poi amenity types where there are less than 10 of that type
clean_pois = pois.groupby('amenity').filter(lambda x: len(x) >= 10)

In [37]:
clean_pois.head()

direction                    geometry     amenity  \
element_type osmid                                                         
node         189770160       NaN  POINT (-73.66345 45.54186)  restaurant   
             190646563       NaN  POINT (-73.66300 45.54201)    post_box   
             191024110       NaN  POINT (-73.67232 45.54761)  restaurant   
             191024149       NaN  POINT (-73.67179 45.54839)  restaurant   
             191024164       NaN  POINT (-73.67196 45.54923)  restaurant   

                                                        name    created_by  \
element_type osmid                                                           
node         189770160                         GP Restaurant           NaN   
             190646563                                   NaN  Potlatch 0.6   
             191024110                            Pizza Roni          JOSM   
             191024149                           La Crèmière           NaN   
             191024164  Le Palais du Chausson et de la Pizza           NaN   

                       addr:city addr:housenumber addr:postcode addr:province  \
element_type osmid                                                              
node         189770160       NaN              NaN           NaN           NaN   
             190646563       NaN              NaN           NaN           NaN   
             191024110       NaN              NaN           NaN           NaN   
             191024149       NaN              NaN           NaN           NaN   
             191024164       NaN              NaN           NaN           NaN   

                       addr:street brand brand:wikidata operator  \
element_type osmid                                                 
node         189770160         NaN   NaN            NaN      NaN   
             190646563         NaN   NaN            NaN      NaN   
             191024110         NaN   NaN            NaN      NaN   
             191024149         NaN   NaN            NaN      NaN   
             191024164         NaN   NaN            NaN      NaN   

                       operator:wikidata automatic_door  \
element_type osmid                                        
node         189770160               NaN            NaN   
             190646563               NaN            NaN   
             191024110               NaN            NaN   
             191024149               NaN            NaN   
             191024164               NaN            NaN   

                       check_date:opening_hours entrance opening_hours  \
element_type osmid                                                       
node         189770160                      NaN      NaN           NaN   
             190646563                      NaN      NaN           NaN   
             191024110                      NaN      NaN           NaN   
             191024149                      NaN      NaN           NaN   
             191024164                      NaN      NaN           NaN   

                       toilets:wheelchair wheelchair source brand:wikipedia  \
element_type osmid                                                            
node         189770160                NaN        NaN    NaN             NaN   
             190646563                NaN        NaN    NaN             NaN   
             191024110                NaN        NaN    NaN             NaN   
             191024149                NaN        NaN    NaN             NaN   
             191024164                NaN        NaN    NaN             NaN   

                       is_in:city name:en old_name phone website addr:unit  \
element_type osmid                                                           
node         189770160        NaN     NaN      NaN   NaN     NaN       NaN   
             190646563        NaN     NaN      NaN   NaN     NaN       NaN   
             191024110        NaN     NaN      NaN   NaN     NaN       NaN   
             19102414

In [38]:
len(clean_pois)

25460

In [ ]:
# put amenities is new categories so we have a lower number of categories

In [45]:
# Initialize an empty list to hold the values for the new column
amenity_cat = []

# Iterate over each value in the 'amenity' column
for x in clean_pois['amenity']:
    # Check conditions and assign corresponding category to amenity_cat list
    if x == 'restaurant' or x == 'cafe' or x == 'fast_food':
        amenity_cat.append('eatery') # edit name of category
    else:
        amenity_cat.append('other') # leave as blank rather than other

# Assign the populated list to the new column 'amenity_cat'
clean_pois['amenity_cat'] = amenity_cat



In [46]:
clean_pois.head()

direction                    geometry     amenity  \
element_type osmid                                                         
node         189770160       NaN  POINT (-73.66345 45.54186)  restaurant   
             190646563       NaN  POINT (-73.66300 45.54201)    post_box   
             191024110       NaN  POINT (-73.67232 45.54761)  restaurant   
             191024149       NaN  POINT (-73.67179 45.54839)  restaurant   
             191024164       NaN  POINT (-73.67196 45.54923)  restaurant   

                                                        name    created_by  \
element_type osmid                                                           
node         189770160                         GP Restaurant           NaN   
             190646563                                   NaN  Potlatch 0.6   
             191024110                            Pizza Roni          JOSM   
             191024149                           La Crèmière           NaN   
             191024164  Le Palais du Chausson et de la Pizza           NaN   

                       addr:city addr:housenumber addr:postcode addr:province  \
element_type osmid                                                              
node         189770160       NaN              NaN           NaN           NaN   
             190646563       NaN              NaN           NaN           NaN   
             191024110       NaN              NaN           NaN           NaN   
             191024149       NaN              NaN           NaN           NaN   
             191024164       NaN              NaN           NaN           NaN   

                       addr:street brand brand:wikidata operator  \
element_type osmid                                                 
node         189770160         NaN   NaN            NaN      NaN   
             190646563         NaN   NaN            NaN      NaN   
             191024110         NaN   NaN            NaN      NaN   
             191024149         NaN   NaN            NaN      NaN   
             191024164         NaN   NaN            NaN      NaN   

                       operator:wikidata automatic_door  \
element_type osmid                                        
node         189770160               NaN            NaN   
             190646563               NaN            NaN   
             191024110               NaN            NaN   
             191024149               NaN            NaN   
             191024164               NaN            NaN   

                       check_date:opening_hours entrance opening_hours  \
element_type osmid                                                       
node         189770160                      NaN      NaN           NaN   
             190646563                      NaN      NaN           NaN   
             191024110                      NaN      NaN           NaN   
             191024149                      NaN      NaN           NaN   
             191024164                      NaN      NaN           NaN   

                       toilets:wheelchair wheelchair source brand:wikipedia  \
element_type osmid                                                            
node         189770160                NaN        NaN    NaN             NaN   
             190646563                NaN        NaN    NaN             NaN   
             191024110                NaN        NaN    NaN             NaN   
             191024149                NaN        NaN    NaN             NaN   
             191024164                NaN        NaN    NaN             NaN   

                       is_in:city name:en old_name phone website addr:unit  \
element_type osmid                                                           
node         189770160        NaN     NaN      NaN   NaN     NaN       NaN   
             190646563        NaN     NaN      NaN   NaN     NaN       NaN   
             191024110        NaN     NaN      NaN   NaN     NaN       NaN   
             19102414

In [31]:
# Replace the file path with the correct path to your JSON file
with open('montreal_park_data.json') as f:
    park_data = json.load(f)

# Create a GeoDataFrame from the features in the JSON data
parks_gdf = gpd.GeoDataFrame.from_features(data)

FileNotFoundError: [Errno 2] No such file or directory: 'montreal_park_data.json'

In [32]:
import os
print(os.getcwd())


/home/jovyan/work/Documents/Social Data Science - Masters/Semester Four/Geospatial Data Science


In [33]:
# Replace 'montreal_park_data.json' with the correct path to your JSON file
parks_gdf = gpd.read_file('montreal_park_data.json')

DriverError: montreal_park_data.json: No such file or directory